## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [18]:
import pandas as pd

In [21]:
ratings = pd.read_csv('ratings.csv')
new = ratings.groupby('userId').count().reset_index()
new = new[['userId', 'rating' ]]
count = new[new['rating']>100]
count.head()

,userId,rating
1,2,184
2,3,656
3,4,242
4,5,101
7,8,155


In [26]:
r_max=ratings.groupby('userId').max().reset_index()
r_min=ratings.groupby('userId').min().reset_index()

In [27]:
max_timestamp=rating_max[['userId','timestamp']]
min_timestamp=rating_min[['userId','timestamp']]
max_timestamp.head()

,userId,timestamp
0,1,1147880055
1,2,1141418008
2,3,1566091883
3,4,1574195101
4,5,858628421


In [34]:
max_timestamp.rename(columns={'timestamp': 'timestamp_max'}, inplace=True)
min_timestamp.rename(columns={'timestamp': 'timestamp_min'}, inplace=True)

In [39]:
timestamp=count.merge(max_timestamp, how='inner').merge(min_timestamp, how='inner')
timestamp.head()

,userId,rating,timestamp_max,timestamp_min
0,2,184,1141418008,1141415509
1,3,656,1566091883,1439472199
2,4,242,1574195101,1573937091
3,5,101,858628421,830786155
4,8,155,891279691,890489203


In [42]:
def difference_max_min(row):
    difference=(row['timestamp_max']-row['timestamp_min'])
    return difference

In [49]:
timestamp['difference']=avg_timestamp.apply(difference_max_min, axis=1)


In [51]:
timestamp.head()

,userId,rating,timestamp_max,timestamp_min
0,2,184,1141418008,1141415509
1,3,656,1566091883,1439472199
2,4,242,1574195101,1573937091
3,5,101,858628421,830786155
4,8,155,891279691,890489203


## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [1]:
import pandas as pd

In [4]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [5]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [6]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [7]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [12]:
merged_table = rzd.merge(auto, how='outer', on='client_id')
merged_table.head()

,client_id,rzd_revenue,auto_revenue
0,111,1093.0,NaN
1,112,2810.0,NaN
2,113,10283.0,57483.0
3,114,5774.0,83.0
4,115,981.0,912.0


In [13]:
merged_table=merged_table.merge(air, how='outer', on='client_id')
merged_table.head()

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0


In [15]:
result_table=merged_table.merge(client_base, how='outer', on='client_id')
result_table.head()

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1


## Задание 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

- необходимо внести данные с номером визита, тк один пользователь может совершать покупку несколько раз, при этом каждый раз имея разное месторасположение
- сопоставить номер визита с координатами в таблице
- из набора координат можно сделать вывод: откуда чаще совершаются покупки, какие покупки в большинстве случаев совершаются из определенного района и тд.